In [1]:
!pip install -q transformers sentencepiece fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 2.2MB 18.3MB/s 
     |████████████████████████████████| 1.2MB 55.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 747kB 50.1MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 870kB 50.0MB/s 
     |████████████████████████████████| 3.3MB 54.3MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 10.1MB 52.0MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 


In [2]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-python

     |████████████████████████████████| 112kB 16.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.19.0-cp37-cp37m-linux_x86_64.whl size=300834 sha256=457de76ab41ac9799b8ff3f60f6dfe7c74c3298ea9c588ae19727f0d4cd005e3
  Stored in directory: /root/.cache/pip/wheels/57/c1/11/2f4306f1990852c5449a220065b1ea587a7a76f3fad8bbe275
Successfully built tree-sitter
Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 2375 (delta 60), reused 83 (delta 33), pack-reused 2256
Receiving objects: 100% (2375/2375), 18.26 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (1489/1489), done.


In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask_finetune"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask_finetune", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [4]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-python']
)

PYTHON_LANGUAGE = Language('build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PYTHON_LANGUAGE)

In [5]:
def get_string_from_code(node, lines, code_list):
	line_start = node.start_point[0]
	line_end = node.end_point[0]
	char_start = node.start_point[1]
	char_end = node.end_point[1]
	if line_start != line_end:
		code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
	else:
		code_list.append(lines[line_start][char_start:char_end])

def my_traverse(code, node, code_list):
	lines = code.split('\n')
	if node.child_count == 0:
		get_string_from_code(node, lines, code_list)
	elif node.type == 'string':
		get_string_from_code(node, lines, code_list)
	else:
		for n in node.children:
			my_traverse(code, n, code_list)
	return ' '.join(code_list)

In [6]:
def gen_comment(method):
  tree = parser.parse(bytes(method, "utf8"))
  code_list=[]
  tokenized_code = my_traverse(method, tree.root_node, code_list)
  out = pipeline([tokenized_code])
  comment = out[0]['summary_text']
  #final_code = f"# {out[0]['summary_text']}\n{method}"
  return comment

In [ ]:
import nest_asyncio
import uvicorn

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok

class Message(BaseModel):
    code: str

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.post('/')
async def root(message: Message):
    cmt = gen_comment(message.code)
    return cmt

url = ngrok.connect(port=80)
print('Public URL:', url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

Public URL: NgrokTunnel: "http://b631c1d73e70.ngrok.io" -> "http://localhost:80"
